1. Described the scenarios where multithreading is preferable to multiprocessing and scenarios where multiprocessing is a better choice.

Multithreading:
Best Scenarios for Multithreading:

I/O-bound tasks: These are tasks where the program spends a lot of time waiting for I/O operations to complete, such as
 network requests, reading from disk, and database queries. Multithreading is well-suited as threads can continue executing other tasks while waiting for I/O.


In [1]:
import threading
import time
import requests

# Simulate an I/O-bound task
def fetch_url(url):
    print(f"Starting download: {url}")
    response = requests.get(url)
    print(f"Finished download: {url}, Status Code: {response.status_code}")

def multithreading_example():
    # List of URLs to fetch
    urls = ["https://www.google.com", "https://www.github.com", "https://www.python.org"]

    threads = []

    # Create and start threads
    for url in urls:
        thread = threading.Thread(target=fetch_url, args=(url,))
        thread.start()
        threads.append(thread)

    # Wait for all threads to finish
    for thread in threads:
        thread.join()

if __name__ == "__main__":
    start_time = time.time()
    multithreading_example()
    print(f"Time taken with threading: {time.time() - start_time:.2f} seconds")


Starting download: https://www.google.comStarting download: https://www.github.com

Starting download: https://www.python.org
Finished download: https://www.python.org, Status Code: 200
Finished download: https://www.google.com, Status Code: 200
Finished download: https://www.github.com, Status Code: 200
Time taken with threading: 0.24 seconds


Multiprocessing:
Best Scenarios for Multiprocessing:

CPU-bound tasks: These are tasks that require significant CPU processing power, such as mathematical computations, data processing, image manipulation, etc. Multiprocessing allows true parallelism, bypassing Python's Global Interpreter Lock (GIL) and taking advantage of multiple CPU cores.

In [2]:
import multiprocessing
import time

# Simulate a CPU-bound task
def compute_fibonacci(n):
    if n <= 1:
        return n
    else:
        return compute_fibonacci(n - 1) + compute_fibonacci(n - 2)

def multiprocessing_example():
    numbers = [35, 36, 37]  # Fibonacci numbers to calculate
    with multiprocessing.Pool(processes=3) as pool:
        results = pool.map(compute_fibonacci, numbers)

    print("Results:", results)

if __name__ == "__main__":
    start_time = time.time()
    multiprocessing_example()
    print(f"Time taken with multiprocessing: {time.time() - start_time:.2f} seconds")


Results: [9227465, 14930352, 24157817]
Time taken with multiprocessing: 24.31 seconds


Comparison:
Scenario 1: I/O-bound task (e.g., HTTP requests)
Multithreading is better for this as I/O operations (like HTTP requests) involve waiting for data from an external resource (such as a server), during which the CPU can be free to handle other tasks. Threads can continue working while some are blocked, thus improving efficiency.
Scenario 2: CPU-bound task (e.g., Fibonacci computation)
Multiprocessing is the better choice for CPU-bound tasks like the Fibonacci computation example. In this case, the work is CPU-intensive, and processes can truly run in parallel, taking advantage of multiple CPU cores, which can result in a significant speedup as compared to multithreading.

2. Described what a process pool is and how it helps in managing a multiple processes efficiently.

A process pool is a collection of worker processes that can execute tasks concurrently. It is particularly useful for parallelizing CPU-bound tasks by distributing work across multiple CPU cores, bypassing the limitations of Python’s Global Interpreter Lock (GIL). The multiprocessing.Pool class in Python provides a simple way to manage a pool of processes, reusing worker processes to handle multiple tasks efficiently.

Instead of creating and destroying processes for each task, a pool creates a fixed number of worker processes that remain active, awaiting tasks. When a task is submitted, it is assigned to an available worker. This reduces the overhead of process creation and management, improving performance, especially for large numbers of tasks.

In [3]:
import multiprocessing

def compute_square(n):
    return n * n

def main():
    with multiprocessing.Pool(processes=4) as pool:
        numbers = [3, 4, 5, 6, 7, 8, 9, 10]
        results = pool.map(compute_square, numbers)
    print(results)

if __name__ == "__main__":
    main()


[9, 16, 25, 36, 49, 64, 81, 100]


3. Explain what muiltiprocessing is and why it is used in Python Programmes.

**Multiprocessing** is a technique in Python that allows multiple processes to run concurrently, each with its own memory space. This is particularly useful for parallelizing **CPU-bound tasks** (tasks that require heavy computation, like number crunching or data processing). Python's **Global Interpreter Lock (GIL)** restricts the execution of Python bytecode to one thread at a time in a single process, limiting the performance of multithreading for CPU-bound tasks. Multiprocessing bypasses this limitation by creating separate processes, each with its own Python interpreter and memory space, allowing true parallelism on multi-core systems.

In Python, the `multiprocessing` module provides classes and functions to create and manage processes. Key benefits of multiprocessing include:

Trune parallism
Multiple processes can run simultaneously on different CPU cores, providing performance improvements for CPU-bound tasks.
Isolation
Each process runs independently, so issues like memory corruption or race conditions are avoided.
Maintained efficiency
By utilizing all available CPU cores, multiprocessing helps to fully leverage a multi-core processor, speeding up compute-intensive tasks.

Multiprocessing is used in Python to speed up programs that involve intensive computations, such as simulations, large-scale data processing, or machine learning. It ensures that CPU resources are effectively utilized, resulting in faster execution times and better performance for parallel tasks.

4. Write a Python Programmeusing multithreading where one thread adds number to a list, and another thread removes number from the list. Implement a mechanism to avoid race condition using threading.Lock.

In [4]:
import threading
import time

# Shared list
shared_list = []

# Create a Lock object to prevent race conditions
lock = threading.Lock()

# Function to add numbers to the list
def add_to_list():
    for i in range(5):
        time.sleep(0.1)  # Simulate some processing time
        with lock:  # Acquire the lock before modifying the shared list
            shared_list.append(i)
            print(f"Added {i} to the list. Current list: {shared_list}")

# Function to remove numbers from the list
def remove_from_list():
    for _ in range(5):
        time.sleep(0.2)  # Simulate some processing time
        with lock:  # Acquire the lock before modifying the shared list
            if shared_list:
                removed = shared_list.pop(0)
                print(f"Removed {removed} from the list. Current list: {shared_list}")
            else:
                print("List is empty. Cannot remove item.")

# Create threads for adding and removing from the list
thread1 = threading.Thread(target=add_to_list)
thread2 = threading.Thread(target=remove_from_list)

# Start both threads
thread1.start()
thread2.start()

# Wait for both threads to finish
thread1.join()
thread2.join()

print("Final shared list:", shared_list)


Added 0 to the list. Current list: [0]
Removed 0 from the list. Current list: []
Added 1 to the list. Current list: [1]
Added 2 to the list. Current list: [1, 2]
Removed 1 from the list. Current list: [2]
Added 3 to the list. Current list: [2, 3]
Added 4 to the list. Current list: [2, 3, 4]
Removed 2 from the list. Current list: [3, 4]
Removed 3 from the list. Current list: [4]
Removed 4 from the list. Current list: []
Final shared list: []


5. Describe the tools and methods avaliable in Python for safely sharing data between threads and processes.

Threading (Multithreading)
Purpose: A Lock is used to ensure that only one thread can access shared data at a time, preventing race conditions.

In [5]:
import threading
lock = threading.Lock()
shared_data = []

def add_to_list():
    with lock:
        shared_data.append(1)


queue.Queue
Purpose: A thread-safe queue for exchanging data between threads, especially useful in producer-consumer scenarios.



In [6]:
import threading
from queue import Queue

queue = Queue()

def producer():
    queue.put('item')

def consumer():
    item = queue.get()
    print(f"Consumed: {item}")


Multiprocessing (Processes)
multiprocessing.Lock
Similar to threading.Lock, the multiprocessing.Lock ensures that only one process accesses shared data at a time.

In [7]:
import multiprocessing
lock = multiprocessing.Lock()
shared_data = []

def add_to_list():
    with lock:
        shared_data.append(1)


multiprocessing.Queue
A process-safe queue used for inter-process communication, allowing data to be safely passed between processes.

In [8]:
import multiprocessing

def producer(queue):
    queue.put('item')

def consumer(queue):
    item = queue.get()
    print(f"Consumed: {item}")

if __name__ == '__main__':
    queue = multiprocessing.Queue()
    p1 = multiprocessing.Process(target=producer, args=(queue,))
    p2 = multiprocessing.Process(target=consumer, args=(queue,))
    p1.start()
    p2.start()
    p1.join()
    p2.join()


Consumed: item


6. Discuss why its's crucial to handle exceptions in concurrent programmes and the techniques avaliable for doing so.

Handling exceptions in concurrent programs is crucial as it unhandled errors can disrupt multiple threads or processes, leading to instability, resource leakage, and incorrect behavior. In multi-threaded and multi-process environments, exceptions in one thread or process can propagate unpredictably, affecting the entire system. Techniques for handling exceptions include using `try-except` blocks within each thread/process, `concurrent.futures` to capture exceptions from worker threads/processes, `Queue` for passing errors between processes, and `logging` for tracking errors. Additionally, `try-finally` blocks ensure proper resource cleanup even if an exception occurs, maintaining program stability and resource integrity.

7. Create a programme that uses a thread pool to calculate the factorial of numbers from 1 to 10 concurrently. Use concurrent.futures.ThreadPoolExecutor to manage the threads.

In [9]:
import concurrent.futures
import math

# Function to calculate the factorial
def calculate_factorial(n):
    return math.factorial(n)

def main():
    # List of numbers from 1 to 10
    numbers = list(range(1, 11))

    # Use ThreadPoolExecutor to manage the threads
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Map the calculate_factorial function to the numbers list
        results = list(executor.map(calculate_factorial, numbers))

    # Print the results
    for num, fact in zip(numbers, results):
        print(f"Factorial of {num} is {fact}")

if __name__ == "__main__":
    main()


Factorial of 1 is 1
Factorial of 2 is 2
Factorial of 3 is 6
Factorial of 4 is 24
Factorial of 5 is 120
Factorial of 6 is 720
Factorial of 7 is 5040
Factorial of 8 is 40320
Factorial of 9 is 362880
Factorial of 10 is 3628800


8. Create a Python Programme that used multiprocessing.Pool to compute the square of numbers from 1 to 10 in parallel. Measure the time taken to perform this computation using a pool of different sizes

In [10]:
import multiprocessing
import time

# Function to calculate square of a number
def calculate_square(n):
    return n * n

# Function to measure time for different pool sizes
def compute_squares(pool_size):
    with multiprocessing.Pool(pool_size) as pool:
        start_time = time.time()
        results = pool.map(calculate_square, range(1, 11))  # Compute squares for numbers 1 to 10
        end_time = time.time()

        print(f"Pool size: {pool_size} -> Results: {results}")
        print(f"Time taken: {end_time - start_time:.6f} seconds\n")
        return end_time - start_time

def main():
    # Different pool sizes to test
    pool_sizes = [1, 2, 4, 8]

    for pool_size in pool_sizes:
        compute_squares(pool_size)

if __name__ == "__main__":
    main()


Pool size: 1 -> Results: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
Time taken: 0.014830 seconds

Pool size: 2 -> Results: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
Time taken: 0.009228 seconds

Pool size: 4 -> Results: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
Time taken: 0.003078 seconds

Pool size: 8 -> Results: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
Time taken: 0.002818 seconds

